<pre>
 ____   ____    _    ____  _____                          _      _     
|  _ \ / ___|  / \  / ___|| ____|     _ __ ___   ___   __| | ___| |___ 
| | | | |     / _ \ \___ \|  _| _____| '_ ` _ \ / _ \ / _` |/ _ \ / __|
| |_| | |___ / ___ \ ___) | |__|_____| | | | | | (_) | (_| |  __/ \__ \
|____/ \____/_/   \_\____/|_____|    |_| |_| |_|\___/ \__,_|\___|_|___/
                                                                        
</pre>

# DCASE-models Notebooks
Python Notebooks for [DCASE-models](https://github.com/pzinemanas/DCASE-models)

---
### About

This notebook shows how to train a model using [DCASE-models](https://github.com/pzinemanas/DCASE-models).

### Overview

The dataset used is [Urban SED](http://urbansed.weebly.com/), an adaptation of the Convolutional Neural Network (CNN) proposed by Salamon and Bello [[SB-CNN]](http://ieeexplore.ieee.org/document/7829341/) is trained end evaluated.

For details on how to download a dataset, extract features or perform data augmentation refer to the respective notebooks.


### Organization

The Notebook is organized into the following sections.
* [1. Prepare data](#Data)
* [2. Initialize model](#InitModel)
* [3. Train model](#TrainModel)
* [4. Evaluate model](#Evaluate)

In [2]:
%load_ext autoreload
%autoreload 2
rootdir_path = '../../'
import sys
import os
import json
import warnings
sys.path.append(rootdir_path)

from dcase_models.data.datasets import URBAN_SED
from dcase_models.data.features import MelSpectrogram
from dcase_models.model.models import SB_CNN_SED
from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.data_augmentation import AugmentedDataset
from dcase_models.data.scaler import Scaler
from dcase_models.util.files import load_json
from dcase_models.util.files import mkdir_if_not_exists, save_pickle
from dcase_models.util.data import evaluation_setup

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id="Data"></a>
## 1. Prepare data

Many of the following steps could be  ommited if *feature_extraction* and *download_and_prepare_datasets* have already been run. Nonetheless, they have been included to make the notebook self contained. 

In [8]:
#load parameters from json file
parameters_file = os.path.join(rootdir_path, 'parameters.json')
params = load_json(parameters_file)
params_dataset = params['datasets']['URBAN_SED']
params_features = params['features']
params_train = params['train']
params_model = params['models']['SB_CNN_SED']

#print dataset parmeters
print(json.dumps(params_features, indent=4, sort_keys=True))
# print feature extraction parameters 
print(json.dumps(params_features, indent=4, sort_keys=True))
# print training parameters 
print(json.dumps(params_features, indent=4, sort_keys=True))

features =  MelSpectrogram(sequence_time=params_features['sequence_time'],
                           sequence_hop_time=params_features['sequence_hop_time'], 
                           audio_win=params_features['audio_win'], 
                           audio_hop=params_features['audio_hop'],  
                           sr=params_features['sr'], 
                           **params_features['MelSpectrogram'])

dataset_path = os.path.join(rootdir_path, params_dataset["dataset_path"])
dataset = URBAN_SED(dataset_path, sequence_hop_time=params_features['sequence_hop_time'])

if dataset.check_if_downloaded():
    print('URBAN_SED dataset was already downloaded. ')
else:
    print('downloading URBAN_SED dataset')
    dataset.download()
# Extract features
if features.check_if_extracted(dataset):
    print('Features were already extracted for URBAN_SED dataset. ')
else:
    print('Extracting features ...')
    features.extract(dataset)


{
    "MelSpectrogram": {
        "mel_bands": 64,
        "n_fft": 1024
    },
    "Openl3": {
        "content_type": "env",
        "embedding_size": 512,
        "input_repr": "mel256"
    },
    "Spectrogram": {
        "n_fft": 1024
    },
    "audio_hop": 690,
    "audio_win": 1024,
    "sequence_hop_time": 1.0,
    "sequence_time": 2.0,
    "sr": 22050
}
{
    "MelSpectrogram": {
        "mel_bands": 64,
        "n_fft": 1024
    },
    "Openl3": {
        "content_type": "env",
        "embedding_size": 512,
        "input_repr": "mel256"
    },
    "Spectrogram": {
        "n_fft": 1024
    },
    "audio_hop": 690,
    "audio_win": 1024,
    "sequence_hop_time": 1.0,
    "sequence_time": 2.0,
    "sr": 22050
}
{
    "MelSpectrogram": {
        "mel_bands": 64,
        "n_fft": 1024
    },
    "Openl3": {
        "content_type": "env",
        "embedding_size": 512,
        "input_repr": "mel256"
    },
    "Spectrogram": {
        "n_fft": 1024
    },
    "audio_hop": 690,
  

Initialise data generators and fit scaler.

In [9]:
# Get train/test folds
folds_train, folds_val, folds_test = evaluation_setup('test', dataset.fold_list,\
                                             params_dataset['evaluation_mode'],
                                             use_validate_set=True)
#initialise train Data Generator
data_gen_train = DataGenerator(dataset, features, folds=folds_train,\
                               batch_size=params_train['batch_size'],
                               shuffle=True, train=True, scaler=None)
# fit scaler
scaler = Scaler(normalizer=params_model['normalizer'])
print('Fitting scaler ...')
scaler.fit(data_gen_train)
print('Done!')

data_gen_train.set_scaler(scaler)

#Initialise validation data Generator

data_gen_val = DataGenerator(dataset, features, folds=folds_val,\
                             batch_size=params_train['batch_size'],
                             shuffle=False, train=False, scaler=scaler)

Fitting scaler ...
Done!


<a id="Model"></a>
## 2. Define Model

In [10]:
params_model = params['models']['SB_CNN_SED']
metrics = ['sed']
X, y = data_gen_train.get_data_batch(0)
 
n_frames_cnn = X.shape[1]
n_freq_cnn = X.shape[2]
n_classes = y.shape[1]

metrics = ['sed']

model_container = SB_CNN_SED(model=None, model_path=None, n_classes=n_classes, 
                             n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn,
                             metrics=metrics, **params_model['model_arguments'])

model_container.model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64)            0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 60, 64)        1664      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 64)        102464    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 64)        0         
__________

<a id="Train"></a>
## 3. Train

In [ ]:
exp_folder='./output/basics'
if not os.path.exists(exp_folder):
    os.makedirs(exp_folder)
model_container.train(data_gen_train, data_gen_val,
                      label_list=dataset.label_list,
                      weights_path=exp_folder, **params['train'],
                      sequence_time_sec=params_features['sequence_hop_time'])

Epoch 1/50
188/188 [==============================] - 11s 61ms/step - loss: 3.0548
F1 = 0.3279, ER = 3.3611 - Best val F1: 0.3279
                  (IMPROVEMENT, saving)

Epoch 2/50
188/188 [==============================] - 11s 57ms/step - loss: 2.9846
F1 = 0.2978, ER = 3.6745 - Best val F1: 0.3279 (0)

Epoch 3/50
188/188 [==============================] - 11s 57ms/step - loss: 2.9377
F1 = 0.3040, ER = 3.4161 - Best val F1: 0.3279 (0)

Epoch 4/50
188/188 [==============================] - 11s 56ms/step - loss: 2.8878
F1 = 0.3298, ER = 2.0810 - Best val F1: 0.3298
                  (IMPROVEMENT, saving)

Epoch 5/50
188/188 [==============================] - 11s 57ms/step - loss: 2.8412
F1 = 0.3837, ER = 1.6204 - Best val F1: 0.3837
                  (IMPROVEMENT, saving)

Epoch 6/50
188/188 [==============================] - 11s 57ms/step - loss: 2.7848
F1 = 0.4215, ER = 1.1327 - Best val F1: 0.4215
                  (IMPROVEMENT, saving)

Epoch 7/50
188/188 [==========================

<a id="Evaluate"></a>
## 4. Evaluate

In [6]:
# Load best_weights
model_container.load_model_weights(exp_folder)
data_gen_test = DataGenerator(dataset, features, folds=folds_test,\
                              batch_size=params_train['batch_size'],\
                              shuffle=False, train=False, scaler=scaler)

kwargs = {'sequence_time_sec': params_features['sequence_hop_time'],
          'metric_resolution_sec': 1.0}
results = model_container.evaluate(data_gen_test, label_list=dataset.label_list, **kwargs)

print(results[metrics[0]])

Segment based metrics
  Evaluated length                  : 20000.00 sec
  Evaluated files                   : 2000 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 14.70 %
    Precision                       : 7.93 %
    Recall                          : 99.77 %
  Error rate
    Error rate (ER)                 : 11.58 
    Substitution rate               : 0.00 
    Deletion rate                   : 0.00 
    Insertion rate                  : 11.58 
  Accuracy
    Sensitivity                     : 99.77 %
    Specificity                     : 0.32 %
    Balanced accuracy               : 50.05 %
    Accuracy                        : 8.20 %

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 14.69 %
    Precision                       : 7.93 %
    Recall                          : 99.77 %
  Error rate
    Error rate (ER)                 : 11.65 
    Deletion ra